## Delete existing Docker images

In [ ]:
source ../config.sh

set -e

TOKEN=$(curl -s -H "Content-Type: application/json" -X POST -d '{"username": "'${DOCKERHUB_USERNAME}'", "password": "'${DOCKERHUB_PASSWORD}'"}' https://hub.docker.com/v2/users/login/ | jq -r .token)

REPOSITORY_NAME="account-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="confirmation-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="payment-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="dotnet-core-react-example"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

## Login to Tanzu Mission Control

In [ ]:
source ../config.sh

tmc login --no-configure --name tanzu-demo

## Create clusters

In [ ]:
source ../config.sh
export $TMC_API_TOKEN=m4CAp1ZUOC81J2czsdWUQVRbx8wPiXBiKClUSEmtbsOJ6248OiDlyiZ8NZbmldd8

tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${CICD_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${ALPHA_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${BRAVO_CLUSTER}

## Monitor creation progress

In [ ]:
tmc cluster list --group bush

## Add clusters to kubeconfig file

In [ ]:
source ../config.sh

KUBECONFIG_CICD=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${CICD_CLUSTER} >> ${KUBECONFIG_CICD} && \
KUBECONFIG_ALPHA=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} >> ${KUBECONFIG_ALPHA} && \
KUBECONFIG_BRAVO=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} >> ${KUBECONFIG_BRAVO}

if [ $? -eq 0 ]; then
    KUBECONFIG=${KUBECONFIG_CICD}:${KUBECONFIG_ALPHA}:${KUBECONFIG_BRAVO} kubectl config view --merge --flatten > ~/.kube/config

    rm ${KUBECONFIG_CICD}
    rm ${KUBECONFIG_ALPHA}
    rm ${KUBECONFIG_BRAVO}
fi

## Update cluster permissions

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${ALPHA_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated